In [5]:
import numpy as np
import GEOparse as gp

Download data if it does not exist:

In [7]:
gse = gp.get_GEO(geo="GSE157239", destdir="./")

09-Nov-2020 01:19:14 DEBUG utils - Directory ./ already exists. Skipping.
09-Nov-2020 01:19:14 INFO GEOparse - File already exist: using local version.
09-Nov-2020 01:19:14 INFO GEOparse - Parsing ./GSE157239_family.soft.gz: 
09-Nov-2020 01:19:14 DEBUG GEOparse - DATABASE: GeoMiame
09-Nov-2020 01:19:14 DEBUG GEOparse - SERIES: GSE157239
09-Nov-2020 01:19:14 DEBUG GEOparse - PLATFORM: GPL21572
09-Nov-2020 01:19:17 DEBUG GEOparse - SAMPLE: GSM4759790
09-Nov-2020 01:19:18 DEBUG GEOparse - SAMPLE: GSM4759791
09-Nov-2020 01:19:18 DEBUG GEOparse - SAMPLE: GSM4759792
09-Nov-2020 01:19:18 DEBUG GEOparse - SAMPLE: GSM4759793
09-Nov-2020 01:19:18 DEBUG GEOparse - SAMPLE: GSM4759794
09-Nov-2020 01:19:18 DEBUG GEOparse - SAMPLE: GSM4759795
09-Nov-2020 01:19:18 DEBUG GEOparse - SAMPLE: GSM4759796
09-Nov-2020 01:19:18 DEBUG GEOparse - SAMPLE: GSM4759797
09-Nov-2020 01:19:18 DEBUG GEOparse - SAMPLE: GSM4759798
09-Nov-2020 01:19:18 DEBUG GEOparse - SAMPLE: GSM4759799
09-Nov-2020 01:19:18 DEBUG GEOpars

In [10]:
print("GSM example:")
for gsm_name, gsm in gse.gsms.items():
    break
    print("Name: ", gsm_name)
    print("Metadata:",)
    for key, value in gsm.metadata.items():
        print(" - %s : %s" % (key, ", ".join(value)))
    print ("Table data:",)
    print (gsm.table.head())
    break

print(gsm.table.head())

GSM example:
(5, 3)
